In [ ]:
%load_ext autoreload
%autoreload 2

Please keep this notebook functionally at par with scripts/oncall/generate_report.py AND linearly runnable (no code with duplicated functionality; run all from this cell down should produce a report). Modify the parameters in `single_fitting_cycle` and `plot_trials` per customizations

In [ ]:
import numpy as np
import pandas as pd

import datetime
import copy
import json
import time
import os

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, create_region_csv, write_csv
from utils.create_report import create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials

In [ ]:
t = time.time()
output_folder = f'../../reports/{t}'

## Params

In [ ]:
ktrials = 10
forecast_days = 45

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, train_period=21, val_period=7, num_evals=1000,
    data_from_tracker=False, initialisation='intermediate', model=SEIRHD, 
    smooth_jump=True,
    which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
predictions_dict['m2'] = single_fitting_cycle(
    state, district, train_period=21, val_period=0, num_evals=1000,
    data_from_tracker=False, initialisation='intermediate', model=SEIRHD, 
    smooth_jump=True,
    which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m2']['data_last_date']

## Fitting Loss

### M1 Loss

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss

In [ ]:
predictions_dict['m2']['df_loss']

## Plot Forecasts

In [ ]:
predictions_dict['m2']['forecast'] = plot_forecast(predictions_dict, (state, district), both_forecasts=False, error_bars=True)

trials_processed = forecast_all_trials(predictions_dict, train_fit='m1')
predictions_dict['m1']['trials_processed'] = trials_processed
predictions_dict['m1']['all_trials'] = trials_to_df(trials_processed)
trials_processed = forecast_all_trials(predictions_dict, train_fit='m2')
predictions_dict['m2']['trials_processed'] = trials_processed
predictions_dict['m2']['all_trials'] = trials_to_df(trials_processed)
kforecasts = plot_top_k_trials(
    predictions_dict,
    train_fit='m2',
    trials_processed=predictions_dict['m2']['trials_processed'], 
    k=ktrials,
    which_compartments=[Columns.confirmed, Columns.active])
predictions_dict['m2']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
predictions_dict['m2']['forecast_active_topk'] = kforecasts[Columns.active]


## Create Report


In [ ]:
create_report(predictions_dict, ROOT_DIR=output_folder)
predictions_dict['m1']['all_trials'].to_csv(os.path.join(output_folder, 'm1-trials.csv'))
predictions_dict['m2']['all_trials'].to_csv(os.path.join(output_folder, 'm2-trials.csv'))

## Create and Save Output CSV

In [ ]:
df_output = create_region_csv(predictions_dict, district, 'district', days=forecast_days)

In [ ]:
write_csv(df_output, filename=os.path.join(output_folder, f'output-{t}.csv'))